In [1]:
!nvidia-smi

Thu Apr  8 09:22:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  !pip uninstall tb-nightly tensorboardX tensorboard
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf

import os
import datetime
import tensorflow_datasets as tfds

%load_ext tensorboard

Uninstalling tensorboard-2.4.1:
  Would remove:
    /usr/local/bin/tensorboard
    /usr/local/lib/python3.7/dist-packages/tensorboard-2.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorboard/*
Proceed (y/n)? y
  Successfully uninstalled tensorboard-2.4.1
     |████████████████████████████████| 455.4MB 41kB/s 
     |████████████████████████████████| 4.0MB 47.6MB/s 
     |████████████████████████████████| 6.0MB 40.6MB/s 
     |████████████████████████████████| 471kB 58.0MB/s 
     |████████████████████████████████| 4.0MB 45.2MB/s 
     |████████████████████████████████| 1.2MB 55.4MB/s 
     |████████████████████████████████| 3.9MB 46.4MB/s 
ERROR: tensorflow 2.4.1 requires tensorboard~=2.4, which is not installed.
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10

In [3]:
import pkg_resources

for entry_point in pkg_resources.iter_entry_points('tensorboard_plugins'):
    print(entry_point.dist)

tensorboard 2.4.1
tensorboard-plugin-wit 1.8.0


In [4]:
#!ls -alrt /usr/local/lib/python3.6/dist-packages/tensorboard*
!rm -r /usr/local/lib/python3.6/dist-packages/tensorboardcolab-0.0.22.dist-info

rm: cannot remove '/usr/local/lib/python3.6/dist-packages/tensorboardcolab-0.0.22.dist-info': No such file or directory


In [5]:
print(tf.__version__)

2.6.0-dev20210407


In [6]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [7]:
dataset, info = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', with_info=True)
train_dataset = dataset['train']

Shuffling and writing examples to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0.incompleteWA0UE3/amazon_us_reviews-train.tfrecord


Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0. Subsequent calls will reuse this data.


In [8]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [9]:
print(train_dataset)

<PrefetchDataset shapes: {data: {customer_id: (), helpful_votes: (), marketplace: (), product_category: (), product_id: (), product_parent: (), product_title: (), review_body: (), review_date: (), review_headline: (), review_id: (), star_rating: (), total_votes: (), verified_purchase: (), vine: ()}}, types: {data: {customer_id: tf.string, helpful_votes: tf.int32, marketplace: tf.string, product_category: tf.string, product_id: tf.string, product_parent: tf.string, product_title: tf.string, review_body: tf.string, review_date: tf.string, review_headline: tf.string, review_id: tf.string, star_rating: tf.int32, total_votes: tf.int32, verified_purchase: tf.int64, vine: tf.int64}}>


In [10]:
len(list(train_dataset))

104975

In [11]:
BUFFER_SIZE = 40000
BATCH_SIZE = 196

In [12]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

In [13]:
for reviews in train_dataset.take(2):
  print(reviews)

{'data': {'customer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'51683810'>, 'helpful_votes': <tf.Tensor: shape=(), dtype=int32, numpy=1>, 'marketplace': <tf.Tensor: shape=(), dtype=string, numpy=b'US'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'Mobile_Electronics'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'B007WO4ENG'>, 'product_parent': <tf.Tensor: shape=(), dtype=string, numpy=b'567489042'>, 'product_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Alpatronix AX200 X-tra Strong Portable Mini Capsule Dual Stereo Speaker for iPhone / iPad / Samsung Galaxy / iPod / Smartphone / MP3 or MP4 player / Laptop or Notebook / Desktop / or Tablet PC'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b"These little speakers really sound better than expected. I would not hesitate to purchase them again. They aren't perfect but then again they sound so much better than my laptop speakers .And are very portable . Not to shabby.">, 'review_date'

In [14]:
for reviews in train_dataset.take(10):
  review_text = reviews['data']
  print(review_text.get('review_body').numpy())
  print(review_text.get('star_rating'))
  print(tf.where(review_text.get('star_rating')>3,1,0).numpy())

b"These little speakers really sound better than expected. I would not hesitate to purchase them again. They aren't perfect but then again they sound so much better than my laptop speakers .And are very portable . Not to shabby."
tf.Tensor(5, shape=(), dtype=int32)
1
b"It wasn't read by the iPad as a compatible item. Was used in 2013 Benz to upgrade the plug from the ipad 2 to the new ipad"
tf.Tensor(1, shape=(), dtype=int32)
0
b'I love the product.  The only issue I have is with the little plug in cord which is fragile and the wires began to show with just a little tug.  Other than that, it works, and I use it daily'
tf.Tensor(4, shape=(), dtype=int32)
1
b"I installed this security system on a 3rd-generation Camry.  I am not a professional installer, just a car enthusiast.  I found a wiring diagram for my car online, and followed Viper's included instructions to match up the wires.  It took me about 4 hours to complete the installation (including breaks).  Great instructions from Vipe

In [15]:
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
for _, reviews in train_dataset.enumerate():
  #print(reviews)
  review_text=reviews['data']
  reviews_tokens = tokenizer.tokenize(review_text.get('review_body').numpy())
  vocabulary_set.update(reviews_tokens)

vocab_size = len(vocabulary_set)
vocab_size

73738

In [16]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, lowercase=True, strip_vocab=True)

In [17]:
print(vocabulary_set)

{'blurry', 'PLAYIN', 'replicate', 'H700', 'Marshall', 'Guetta', 'MP4047', 'mecanic', 'transitional', 'Only', 'Swiveling', 'KWAV50', 'Oz', 'rookie', 'Traps', 'Windex', 'Mamouth', 'pefectly', 'Tornado', 'tolerrate', 'B003LAEJ9O', 'nests', 'rebel', 'applicator', 'sos', 'suppository', 'Dennis', 'shortness', 'loner', 'boils', 'RECHARGE', 'SANSA', 'Dio1978', 'WORKMANSHIP', 'crane', 'onlive', 'pubs', 'BALTIMORE', 'Iphone4', 'cats', 'GRACIAS', '7870', 'ShorRS', 'Fulfills', 'ONES', 'Shefko', 'wernt', 'monotonous', 'crinkling', 'STIUCKER', '9600', 'lang', 'INFAMOUS', 'senn', 'irregular', 'maxx', 'Queen', 'ROAD', 'SKar', 'disiponted', 'samlple', 'Grandmas', 'landrover', 'bassknobs', 'Orden', 'dynamically', 'Pandigitgal', 'reasonable', 'advertiser', 'orr', 'perforated', 'AT2020', 'qty', 'nada', 'Cane', '1G', 'sliced', 'protacting', 'rarley', 'Powergen', 'Wort', 'shoveling', 'Sci', 'mande', 'gates', 'purging', 'precluding', 'upgraded', 'marketers', 'faint', 'Removes', 'audis', '207', 'fantasic', 'E

In [18]:
for reviews in train_dataset.take(5):
  review_text = reviews['data']
  print(review_text.get('review_body').numpy())
  encoded_example = encoder.encode(review_text.get('review_body').numpy())
  print(encoded_example)

b"These little speakers really sound better than expected. I would not hesitate to purchase them again. They aren't perfect but then again they sound so much better than my laptop speakers .And are very portable . Not to shabby."
[44778, 59256, 66180, 70060, 64788, 68814, 68250, 39070, 66476, 62628, 41837, 48776, 45613, 49612, 65204, 72004, 55132, 70811, 56317, 66936, 72345, 68733, 72004, 55132, 64788, 69697, 62046, 68814, 68250, 68137, 68058, 66180, 70877, 50704, 45722, 56145, 41837, 45613, 42691]
b"It wasn't read by the iPad as a compatible item. Was used in 2013 Benz to upgrade the plug from the ipad 2 to the new ipad"
[49127, 38498, 56317, 57699, 42195, 54230, 70404, 52931, 63019, 58096, 26914, 34314, 73727, 18732, 25731, 72908, 45613, 44844, 54230, 60709, 66278, 54230, 70404, 70525, 45613, 54230, 51893, 70404]
b'I love the product.  The only issue I have is with the little plug in cord which is fragile and the wires began to show with just a little tug.  Other than that, it works,

In [19]:
for index in encoded_example:
  print('{} ----> {}'.format(index, encoder.decode([index])))

66936 ----> perfect
44117 ----> gift


In [20]:
encoder.save_to_file("/tmp/sa_encoder.vocab")

In [21]:
def encode(text_tensor, label_tensor):
  encoded_text = encoder.encode(text_tensor.numpy())
  label = tf.where(label_tensor>3,1,0)
  return encoded_text, label

In [22]:
def encode_map_fn(tensor):
  
  text = tensor['data'].get('review_body')
  label = tensor['data'].get('star_rating') 

  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int32))

  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label

In [23]:
ar_encoded_data = train_dataset.map(encode_map_fn)

In [24]:
for f0,f1 in ar_encoded_data.take(2):
  print(f0)
  print(f1)

tf.Tensor(
[44778 59256 66180 70060 64788 68814 68250 39070 66476 62628 41837 48776
 45613 49612 65204 72004 55132 70811 56317 66936 72345 68733 72004 55132
 64788 69697 62046 68814 68250 68137 68058 66180 70877 50704 45722 56145
 41837 45613 42691], shape=(39,), dtype=int64)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(
[49127 38498 56317 57699 42195 54230 70404 52931 63019 58096 26914 34314
 73727 18732 25731 72908 45613 44844 54230 60709 66278 54230 70404 70525
 45613 54230 51893 70404], shape=(28,), dtype=int64)
tf.Tensor(0, shape=(), dtype=int32)


In [25]:
TAKE_SIZE=14000

train_data = ar_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = ar_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [26]:

vocab_size += 1

In [27]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]



(<tf.Tensor: shape=(1642,), dtype=int64, numpy=array([44778, 59256, 66180, ...,     0,     0,     0])>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [28]:

for f0,f1 in test_data.take(10):
  print(tf.unique_with_counts(f1)[2].numpy())

[136  60]
[132  64]
[142  54]
[ 71 125]
[120  76]
[127  69]
[ 62 134]
[ 61 135]
[141  55]
[ 69 127]


In [29]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 128))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
for units in [64, 32, 16]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))
  model.add(tf.keras.layers.Dropout(units*0.006))
model.add(tf.keras.layers.Dense(1))


In [30]:
!rm -r /tmp/logs/

rm: cannot remove '/tmp/logs/': No such file or directory


In [31]:
logdir = os.path.join("/tmp/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/tmp/sentiment_analysis.hdf5', verbose=1, save_best_only=True)

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history=model.fit(train_data, epochs=6, validation_data=test_data, callbacks=[tensorboard_callback,checkpointer])

In [ ]:
model.save('/tmp/final_sentiment_analysis.hdf5')

In [ ]:
!ls -alrt /tmp/*.hdf5

In [ ]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

In [ ]:
for f0,f1 in test_data.take(1):
  print(f1)
  print(model.predict(f0))


In [ ]:
model.layers

In [ ]:
model.summary()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.get_layer('embedding').output

In [ ]:
!nvidia-smi

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')

In [ ]:
tf.keras.backend.clear_session()
#sa_load = tf.keras.models.load_model('/tmp/sentiment_analysis.hdf5',compile = False)
sa_load = tf.keras.models.load_model('/tmp/final_sentiment_analysis.hdf5',compile = False)

In [ ]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

In [ ]:
def predict_fn(pred_text):
  encoded_pred_text = encoder.encode(pred_text)
  print(encoded_pred_text)
  encoded_pred_text = pad_to_size(encoded_pred_text, 32)
  print(encoded_pred_text)
  encoded_pred_text = tf.cast(encoded_pred_text, tf.float32)
  predictions = sa_load.predict(tf.expand_dims(encoded_pred_text, 0))

  return (predictions)

In [ ]:
pred_text = ('This watch is pretty bad. Color and dial looks pathetic.')
predictions = predict_fn(pred_text)
print(predictions)

In [ ]:
pred_text = ('Color and dial does not look as advertised. Shipping was not any better as well')
predictions = predict_fn(pred_text)
print(predictions)

In [ ]:
pred_text = ('Amazing product. Fast Delivery. Nice packing')
predictions = predict_fn(pred_text)
print(predictions)

In [ ]:
pred_text = ('Nice product. Packing could have been better')
predictions = predict_fn(pred_text)
print(predictions)

In [ ]:
print(tf.distribute.get_strategy())

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir /tmp/logs

In [ ]:
from tensorboard import notebook
notebook.list()

In [ ]:
notebook.display(port=6006, height=1000) 